In [ ]:
! pip install pytelegrambotapi

     |████████████████████████████████| 104 kB 27.9 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.0.0-py3-none-any.whl size=79300 sha256=50247cc42266fdbbde539c84c25979e62ee9b66409da6b6d48ac9b324539f9f1
  Stored in directory: /root/.cache/pip/wheels/37/0d/f5/48d55810a3b7ba9d3a17d08fad99547bb19988d0bc24836979
Successfully built pytelegrambotapi


In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords

import telebot
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df_all = pd.read_excel('GeneralDataset_v.0.xlsx')

In [ ]:
df_all.head()

,Ques,General_word,Answer
0,Что такое кортеж?,"кортеж, tuple, что",Кортеж это неизменяемый список
1,"Зачем нужны кортежи, если есть списки?","кортеж, список, зачем, tuple","Кортеж защищен от изменений, занимает меньше м..."
2,Как создать кортеж?,"кортеж, создать, tuple",a = tuple()
3,Как создать кортеж с одним элементом?,"кортеж, создать, элемент, tuple","a = tuple('elem', )"
4,Какие операции поддерживают кортежи,"кортеж, операции, tuple",Кортеж поддерживает все операции на


In [ ]:
df_all.drop(labels='General_word', axis=1, inplace=True)

In [ ]:
# функция препроцессинга
def preprocessing(text):
  text = text.lower()
  text = re.sub(r'[-\(\)\"#\/@;:<>\{\}\-=~|\.\?]', '', text)
  text = re.sub(r'\d','',text)
  return text

df_all['Ques'] = df_all['Ques'].apply(preprocessing)


In [ ]:
# Лемматизация и избавление от стоп слов
def lem_stop(text):
  wr_lem = WordNetLemmatizer()
  lemma_text = [wr_lem.lemmatize(word) for word in text.split()]
  stop_nltk = stopwords.words('russian')
  our_stop = ['python', 'питон', 'такое', 'чем', 'почему', 'какие', '']
  stop_nltk += our_stop
  not_stop_word = [word for word in lemma_text if word not in stop_nltk]
  return ' '.join(not_stop_word)


df_all['Ques'] = df_all['Ques'].apply(lem_stop)


In [ ]:
df_all

,Ques,Answer
0,кортеж,Кортеж это неизменяемый список
1,"нужны кортежи, списки","Кортеж защищен от изменений, занимает меньше м..."
2,создать кортеж,a = tuple()
3,создать кортеж одним элементом,"a = tuple('elem', )"
4,операции поддерживают кортежи,Кортеж поддерживает все операции на
...,...,...
532,pyc py,Чем файл .pyc отличается от .py? Оба файла со...
533,объектноориентированный язык программирования,Что делает питон объектно-ориентированным? Он...
534,ооп,Что делает питон объектно-ориентированным? Он...
535,список,Списки в Python - упорядоченные изменяемые кол...


In [ ]:
# Применим tfidf
tfidf = TfidfVectorizer()
vec_tfidf = tfidf.fit_transform(df_all['Ques']).toarray()
df_tfidf = pd.DataFrame(vec_tfidf, columns=tfidf.get_feature_names())
df_tfidf.shape

(537, 520)

In [ ]:
def tfidf_answer(text):
  lemma = preprocessing(text)
  tf = tfidf.transform([lemma]).toarray()
  cos = 1 - pairwise_distances(df_tfidf, tf, metric='cosine')
  value_index = cos.argmax()
  return df_all['Answer'].loc[value_index]

In [ ]:
tfidf_answer('Что такое список?')

'Списки в Python - упорядоченные изменяемые коллекции объектов произвольных типов (почти как массив, но типы могут отличаться).  Чтобы использовать списки, их нужно создать. Создать список можно несколькими способами. Например, можно обработать любой итерируемый объект (например, строку) встроенной функцией list:'

In [ ]:
API_TOKEN = '1938597563:AAEeUO_PFbg0H_kYvPKiXMGQaXXYT0zZFv0'
bot = telebot.TeleBot(API_TOKEN)

def gen_markup():
    markup = InlineKeyboardMarkup()
    markup.row_width = 2
    markup.add(InlineKeyboardButton("Да", callback_data="cb_yes"),
               InlineKeyboardButton("Нет", callback_data="cb_no"))
    return markup


@bot.message_handler(commands=['start'])
def get_text_messages(message):
  if message.text == "/start":
    bot.send_message(message.from_user.id, "Привет {}!\nЯ могу помочь тебе освоить Python.\nПиши свой вопрос, и я подберу для тебя инфу".format(message.from_user.first_name))
  

@bot.message_handler(func=lambda message: True)
def get_answer(message):
  global google_ques, mes
  mes = message 
  google_ques = message.text.replace('?','').split()
  bot.reply_to(message, tfidf_answer(message.text))
  bot.send_message(message.chat.id, "Тебя устроил ответ на вопрос?", reply_markup=gen_markup())


@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    global google_ques, mes
    if call.data == "cb_yes":
        bot.answer_callback_query(call.id, "Рады были помочь! Можешь задать следующий вопрос.")
    elif call.data == "cb_no":
        out = 'https://www.google.com/search?q=' + '+'.join(google_ques) + '+Python?'
        bot.answer_callback_query(call.id, bot.send_message(mes.chat.id, out))   


bot.polling(none_stop=True)